In [1]:
from rich.pretty import pprint

## The DACBO Benchmark
Let's take a look at DACBO. This is a benchmark for controlling hyperparameters of a Bayesian Optimization (BO) loop. First, let's make an instance of the benchmark:

In [ ]:
from dacbench.benchmarks import DACBOBenchmark
bench = DACBOBenchmark()

Now let's take a look at the elements of the config in this benchmark:

In [3]:
pprint(list(bench.config.keys()))

[
│   'reward_range',
│   'seed',
│   'instance_set_path',
│   'observation_keys',
│   'action_space_class',
│   'action_space_kwargs',
│   'reward_keys',
│   'inner_seeds',
│   'benchmark_info'
]

The 'benchmark_info' tells us some things about this benchmark already:

In [4]:
pprint(bench.config["benchmark_info"])

{
│   'identifier': 'DACBO',
│   'name': 'DACBO',
│   'reward': 'Incumbent cost',
│   'state_description': [
│   │   'incumbent_changes',
│   │   'trials_passed',
│   │   'trials_left',
│   │   'ubr',
│   │   'searchspace_dim',
│   │   'continuous_hps',
│   │   'categorical_hps',
│   │   'ordinal_hps',
│   │   'int_hps',
│   │   'tsp',
│   │   'knn_entropy',
│   │   'y_skewness',
│   │   'y_kurtosis',
│   │   'y_mean',
│   │   'y_std',
│   │   'y_variability',
│   │   'tsp_best',
│   │   'knn_entropy_best',
│   │   'y_skewness_best',
│   │   'y_kurtosis_best',
│   │   'y_mean_best',
│   │   'y_std_best',
│   │   'y_variability_best',
│   │   'budget_percentage',
│   │   'inc_improvement_scaled',
│   │   'has_categorical_hps',
│   │   'knn_difference',
│   │   'ubr_difference',
│   │   'acq_value_EI',
│   │   'acq_value_PI',
│   │   'previous_param'
│   ]
}

The reward in this task has the following reward range:

In [5]:
pprint(bench.config["reward_range"])

[-inf, inf]

The config also contains some standard keys like the seed, instance set, list of observation keys, action space class, inner seeds to be used, etc. By default, the agent controls the $\beta$ parameter in the Upper Confidence Bound (UCB) acquisition function of the BO loop. For further configuration details, please refer to the dacboenv package.

## DACBO Instances
Now let's take a look at how a DACBO instance looks. To do so, we first read the default instance set and look at its only element:

In [6]:
pprint(bench.config["instance_set_path"])
bench.read_instance_set()
pprint(bench.config.instance_set[0])

'bbob_2_default.csv'

['bbob/2/1/0', 'bbob/2/20/0']

As in the instance selection is handled internally by the DACBO environment, we only provide a list of target functions to be considered as offered by CARP-S as a single instance. A DACBO instance consists of an inner seed and a target function. By default, 3 inner seeds are chosen at random an the cross product of these seeds and all selected target functions is evaluated in a round robin manner.

## Running DACBO
Lastly, let's look at the DACBO benchmark in action. Because some observations rely on reference incumbent values, we first run SMAC to create a baseline. Additionally, the first BO run's initial design is evaluated upon resetting.

In [7]:
env = bench.get_environment()
pprint(env.reset())

[12:08:01] INFO     Loading performance data from                                      ]8;id=478165;file:///scratch/hpc-prf-intexml/tklenke/repos/dacboenv/dacboenv/utils/reference_performance.py\reference_performance.py]8;;\:]8;id=780670;file:///scratch/hpc-prf-intexml/tklenke/repos/dacboenv/dacboenv/utils/reference_performance.py#243\243]8;;\
                    reference_performance/reference_performance.parquet                                            

[WARNING][target_function_runner.py:74] The argument budget is not set by SMAC: Consider removing it from the target function.
[WARNING][target_function_runner.py:74] The argument instance is not set by SMAC: Consider removing it from the target function.
[WARNING][target_function_runner.py:74] The argument cutoff is not set by SMAC: Consider removing it from the target function.
[INFO][abstract_initial_design.py:139] Using 16 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:307] Using only one seed for deterministic scenario.
[INFO][abstract_intensifier.py:517] Added config 8b5eb2 as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:596] Added config e81dca and rejected config 8b5eb2 as incumbent because it is not better than the incumbents on 1 instances: 
[INFO][abstract_intensifier.py:596] Added config de4ce3 and rejected config e81dca as incumbent because it is not better than the incumbents on 1 instance

(
│   {
│   │   'incumbent_changes': array([0.], dtype=float32),
│   │   'trials_passed': array([0.], dtype=float32),
│   │   'trials_left': array([-1.], dtype=float32),
│   │   'ubr': array([-1.], dtype=float32),
│   │   'searchspace_dim': array([0.], dtype=float32),
│   │   'continuous_hps': array([0.], dtype=float32),
│   │   'categorical_hps': array([0.], dtype=float32),
│   │   'ordinal_hps': array([0.], dtype=float32),
│   │   'int_hps': array([0.], dtype=float32),
│   │   'tsp': array([-1.], dtype=float32),
│   │   'knn_entropy': array([0.], dtype=float32),
│   │   'y_skewness': array([0.], dtype=float32),
│   │   'y_kurtosis': array([0.], dtype=float32),
│   │   'y_mean': array([0.], dtype=float32),
│   │   'y_std': array([-1.], dtype=float32),
│   │   'y_variability': array([-1.], dtype=float32),
│   │   'tsp_best': array([-1.], dtype=float32),
│   │   'knn_entropy_best': array([0.], dtype=float32),
│   │   'y_skewness_best': array([0.], dtype=float32),
│   │   'y_kurtosis_best': array([0.], dtype=float32),
│   │   'y_mean_best': array([0.], dtype=float32),
│   │   'y_std_best': array([-1.], dtype=float32),
│   │   'y_variability_best': array([-1.], dtype=float32),
│   │   'budget_percentage': array([0.], dtype=float32),
│   │   'inc_improvement_scaled': array([0.], dtype=float32),
│   │   'has_categorical_hps': array([0.], dtype=float32),
│   │   'knn_difference': array([0.], dtype=float32),
│   │   'ubr_difference': array([0.], dtype=float32),
│   │   'acq_value_EI': array([0.], dtype=float32),
│   │   'acq_value_PI': array([0.], dtype=float32),
│   │   'previous_param': array([10.], dtype=float32),
│   │   'gp_hp_k1__k1__constant_value0_observation': array([0.], dtype=float32),
│   │   'gp_hp_k1__k2__length_scale0_observation': array([0.], dtype=float32),
│   │   'gp_hp_k1__k2__length_scale1_observation': array([0.], dtype=float32),
│   │   'gp_hp_k2__noise_level0_observation': array([0.], dtype=float32)
│   },
│   {}
)

If we take a step, we see the updated state:

In [8]:
action = env.action_space.sample()
state, reward, terminated, truncated, info = env.step(action)
pprint(state)

[INFO][abstract_intensifier.py:596] Added config 6bb167 and rejected config 46821d as incumbent because it is not better than the incumbents on 1 instances: 
[INFO][dacboenv.py:359] Step: 1, instance: (39590, 'bbob/2/20/0') Reward: -187.08337915468118, terminated: False, truncated: False, info: {}


{
│   'incumbent_changes': array([3.], dtype=float32),
│   'trials_passed': array([17.], dtype=float32),
│   'trials_left': array([60.], dtype=float32),
│   'ubr': array([47989.965], dtype=float32),
│   'searchspace_dim': array([2.], dtype=float32),
│   'continuous_hps': array([2.], dtype=float32),
│   'categorical_hps': array([0.], dtype=float32),
│   'ordinal_hps': array([0.], dtype=float32),
│   'int_hps': array([0.], dtype=float32),
│   'tsp': array([4.407388], dtype=float32),
│   'knn_entropy': array([0.54839134], dtype=float32),
│   'y_skewness': array([2.1103723], dtype=float32),
│   'y_kurtosis': array([4.0757565], dtype=float32),
│   'y_mean': array([35196.844], dtype=float32),
│   'y_std': array([50527.785], dtype=float32),
│   'y_variability': array([1.2443994], dtype=float32),
│   'tsp_best': array([0.], dtype=float32),
│   'knn_entropy_best': array([-0.27582544], dtype=float32),
│   'y_skewness_best': array([0.], dtype=float32),
│   'y_kurtosis_best': array([0.], dtype=float32),
│   'y_mean_best': array([187.08337], dtype=float32),
│   'y_std_best': array([0.], dtype=float32),
│   'y_variability_best': array([2.020312], dtype=float32),
│   'budget_percentage': array([0.22077923], dtype=float32),
│   'inc_improvement_scaled': array([0.01305182], dtype=float32),
│   'has_categorical_hps': array([0.], dtype=float32),
│   'knn_difference': array([0.], dtype=float32),
│   'ubr_difference': array([0.], dtype=float32),
│   'acq_value_EI': array([0.20158173], dtype=float32),
│   'acq_value_PI': array([0.5000079], dtype=float32),
│   'previous_param': array([-0.37298787], dtype=float32),
│   'gp_hp_k1__k1__constant_value0_observation': array([0.46420226], dtype=float32),
│   'gp_hp_k1__k2__length_scale0_observation': array([0.0858638], dtype=float32),
│   'gp_hp_k1__k2__length_scale1_observation': array([-0.5649307], dtype=float32),
│   'gp_hp_k2__noise_level0_observation': array([-25.], dtype=float32)
}

Furthermore, we also get a reward and a truncation signal. Truncation will be set to true after a single BO run has finished and the next instance will be selected internally.

In [9]:
pprint(f"Reward {reward}")
pprint(f"Truncated {truncated}")

'Reward -187.08337915468118'

'Truncated False'